# File to paginate and convert textbook images into the proper format
> Note: This file doesnt actually work on my machine due to some unresolved poppler pdf2image issues - this is just an imported representation of what is done in a similar google colab pipeline

In [6]:
!pip install pdf2image
!pip install poppler-utils

In [2]:
import os
# find directory of textbooks in drive
loc = "full-book-images/"  # input("File Directory?:\n")
textbooks = [file for file in os.listdir(loc) if '.pdf' in file]
print(textbooks)

# from google.colab import drive
# drive.mount('/content/drive')
# import os
# # find directory of textbooks in drive
# loc = "/content/drive/MyDrive/gpt_pipeline/textbooks/"
# textbooks = [file for file in os.listdir(loc) if '.pdf' in file]
# print("Found", len(textbooks), "books:")
# print(textbooks)

['StuartSutherland-The Verilog PLI Handbook.pdf']


In [7]:
from pdf2image import pdfinfo_from_path, convert_from_path

save_dir = "paginated/"
def paginate(dir, name, batch=10, verbose=True):
  # Note: convert_from_path is too much ram for all pages at once
  info = pdfinfo_from_path(dir, userpw=None, poppler_path=None)
  maxPages = 20 #info["Pages"]  # get num pages
  for page_num in range(1, maxPages+1, batch): # load pages by batch
    pages = convert_from_path(dir, dpi=200, first_page=page_num, 
                              last_page=min(page_num+batch-1,maxPages))
    for count, page in enumerate(pages):  # paginate
        saved_at = f'{save_dir}{name}_p{page_num+count}.jpg'
        # page.save(saved_at, 'JPEG')  # convert to jpeg
        if verbose: print("saved at", saved_at)

# run results on found textbooks
for book in textbooks:
  paginate(loc+book, book[:-4], 20)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [ ]:
!pip install openai

In [ ]:
# ! imports
import os
from openai import OpenAI
import base64
from tqdm import tqdm
import pandas as pd
import numpy as np
from IPython import display

cwd = os.getcwd()


# ! main gpt-interacting functions
client = OpenAI(
    # This is the default and can be omitted
    api_key="",  # !!!!
)

def run_gpt(this_messages, this_model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(messages=this_messages,
                                                     model=this_model)
    return chat_completion

def gpt_oneshot(input_prompt, directive="You are a helpful assistant.", verbose=False):
    message_hist = [{"role": "system", "content": directive},  # add directed
                    {"role": "user", "content": input_prompt}]  # init
    response = run_gpt(message_hist)["choices"][0]["message"]["content"]
    if verbose:
        print("chat_gpt: ", response, '\n')
#     message_hist.append({"role": "system", "content": response})
    return response
    
def gpt_image_oneshot(image_link, new_prompt="This image contains some information about Verilog. Please respond with a question\
                                            for which the answer is the code snippet on this page.",
                        verbose=False):
    # "This image contains a block of Verilog code and text relating to it. "+...
    new_content = [{"type": "text", "text": new_prompt},
                    {"type": "image_url", "image_url": {"url": image_link}}]  # "I am the owner/maker of this image", "I am an ML researcher"
    message_hist = [{"role": "system", "content": "You are a helpful assistant that gives information on images of code."},
                    {"role": "user", "content": new_content}]  # init
    if verbose:
        print("Asked:", new_content[0]["text"])
#         print("With Image:", new_content[1]["image_url"]["url"])
    response = run_gpt(message_hist, "gpt-4-vision-preview").choices[0].message.content
    if verbose: print("\nResponded With:", response)
    return response



# helpers
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_code(text):
    if text is None or len(text) == 0:
        return []
    this_code = []
    for i, code_chunk in enumerate(text.split('```')):  # pieces of code are denoted by ``` so we split
        if i%2 == 1:  # every other chunk is a piece of code in this case
            this_code.append(code_chunk[7:])  # Note: code declarations also have 'python\n' denoting the language, since we dont need this, we omit the first 7 chars
    return this_code

indicators = ["```"]
def has_code(message):
    message = message.strip()
    if bad_message in message:
        return False  # instant signal
    for ind in indicators:
        if ind in message:
            return True
    return False



# ! main prompting function
prompt_precursor = "This image contains some information about Verilog. "
bad_message = "NO CODE"
ask_code = prompt_precursor+"If there is some code on this page then please write it. Otherwise say \""+bad_message+"\""
# supervised addition
ask_caption = "This image contains some Verilog code on it. Please give a short caption to describe this code."
# additional queries
ask_question = "This image contains some Verilog code on it. Please respond with a question for which the answer is the code snippet on this page."

def extract_code(image_path, atmps=3, verbose=False):
    """ takes in the link of a local image and gets information about code on it """
    base64_image = encode_image(image_path)
    image_link = f"data:image/jpeg;base64,{base64_image}"
    
    attempts = 0
    raw_code = ""  # get gpt to extract code
    while raw_code == "" and attempts < atmps:
        try:
            raw_code = gpt_image_oneshot(image_link, ask_code)
            attempts = atmps
        except Exception as e:
            attempts += 1
            if verbose: print("Encountered error:", e)
    # if there was some output we save it
    if raw_code == "":
        print("*couldnt get code after", atmps, "attempts")
    else:
        processed_code = ""
        caption = ""
        is_code = has_code(raw_code)
        if is_code:
            processed_code = '\n'.join(get_code(raw_code))  # NOTE: aggregation not necessary
            # * process caption
            attempts = 0
            while caption == "" and attempts < atmps:
                try:
                    caption = gpt_image_oneshot(image_link, ask_caption)
                    attempts = atmps
                except Exception as e:
                    attempts += 1
                    if verbose: print("Encountered error:", e)
            # * process question
            attempts = 0
            question = ""
            while question == "" and attempts < atmps:
                try:
                    question = gpt_image_oneshot(image_link, ask_question)
                    attempts = atmps
                except Exception as e:
                    attempts += 1
                    if verbose: print("Encountered error:", e)
        has_finished = is_code and (caption != "") and (question != "")  # signifier that we have consistent output on both fronts
        return [image_path, has_finished, raw_code, processed_code, caption]

In [ ]:
# find file directories
data_dir = "/content/drive/MyDrive/gpt_pipeline/pages/"
ims = [data_dir+file for file in os.listdir(data_dir) if 'jpg' in file]  # TOOD SORT TO ENSURE STABILITY
print("Found", len(ims), "files\n")

# for partial execution on google and partial on my computer
ims = [file for file in ims if 'book' in file.lower()]
print("For partial execution, allocating", len(ims), "books")

In [ ]:
# process them
results = []
# checkpointing
checkpoint_dir = "/content/drive/MyDrive/gpt_pipeline/checkpoints/textbook-gpt"
delim = 100 if len(ims) > 100 else len(ims)
print("Processing textbooks with checkpoints every", delim)
for i, im in tqdm(enumerate(ims)):
  if i > 9600:  # starting off at last execution (stops at 2.9k, 5.2k)
    results.append(extract_code(im, 2))
    if i > 0 and ((i%delim) == 0 or i >= (len(ims)-1)):  # save partial execution
      df = pd.DataFrame(results, columns=["image_directory", "isComplete", "raw_code", "code", "caption"])
      save_dir = checkpoint_dir+str(i-delim)+"-"+str(i)+".csv"
      df.to_csv(save_dir, index=False)
      print("\nSAVED TO ", save_dir)
      print()
      results = []